In [131]:
!pip install -q numpy pandas scikit-learn openai groq seaborn matplotlib dotenv

### 1. Import Dependencies

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
from enum import Enum
from pydantic import BaseModel
from dotenv import load_dotenv
import openai, groq
import os


### 2. Basic Information

In [133]:
os.getcwd()

'c:\\Users\\Hello\\Desktop\\Projects\\Machine Learning\\ML Course - Real World Applications\\Mini_Project_1\\Telco-Customer-Churn-Prediction\\EDA'

In [134]:
import pandas as pd

# Load Excel file with xlrd engine for .xls files
df = pd.read_csv("Data/Raw/telco-customer-dataset.xls")
df

# # Save to CSV
# df.to_csv("Data/Raw/Raw_Data.csv", index=False)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [135]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [136]:
# Create a mask of space-only cells
mask = df.astype(str).applymap(lambda x: x.strip() == '')

# Get positions
positions = mask[mask].stack().index.tolist()

for row, col in positions:
    print(f'Row {row}, Column "{col}" has a space-only value')

Row 488, Column "TotalCharges" has a space-only value
Row 753, Column "TotalCharges" has a space-only value
Row 936, Column "TotalCharges" has a space-only value
Row 1082, Column "TotalCharges" has a space-only value
Row 1340, Column "TotalCharges" has a space-only value
Row 3331, Column "TotalCharges" has a space-only value
Row 3826, Column "TotalCharges" has a space-only value
Row 4380, Column "TotalCharges" has a space-only value
Row 5218, Column "TotalCharges" has a space-only value
Row 6670, Column "TotalCharges" has a space-only value
Row 6754, Column "TotalCharges" has a space-only value


C:\Users\Hello\AppData\Local\Temp\ipykernel_12968\671002547.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df.astype(str).applymap(lambda x: x.strip() == '')


In [137]:
# Remove all entries with only whitespace and replace them with 0
df['TotalCharges'] = df['TotalCharges'].replace(r'^\s*$', 0, regex=True)


In [138]:
# Create a mask of space-only cells
mask = df.astype(str).applymap(lambda x: x.strip() == '')

# Get positions
positions = mask[mask].stack().index.tolist()

for row, col in positions:
    print(f'Row {row}, Column "{col}" has a space-only value')

C:\Users\Hello\AppData\Local\Temp\ipykernel_12968\671002547.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df.astype(str).applymap(lambda x: x.strip() == '')


### 3. Replacing Yes and No with 1 and 0

In [139]:

df['Partner'] = df['Partner'].apply(lambda x: 1 if x=="Yes" else 0)
df['Dependents'] = df['Dependents'].apply(lambda x: 1 if x=="Yes" else 0)
df['PhoneService'] = df['PhoneService'].apply(lambda x: 1 if x=="Yes" else 0)
df['PaperlessBilling'] = df['PaperlessBilling'].apply(lambda x: 1 if x=="Yes" else 0)
df['Churn'] = df['Churn'].apply(lambda x: 1 if x=="Yes" else 0)

# create a new feature called 'HasInternet' based on the 'InternetService' column
df['HasInternet'] = (df['InternetService'] != 'No').astype(int)

#change the no internet service value to no for one hot encoding
internet_related_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

for col in internet_related_cols:
    df[col] = df[col].replace('No internet service', 'No')

# List of columns to encode
binary_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Map Yes → 1, No → 0
yes_no_map = {'Yes': 1, 'No': 0}

for col in binary_cols:
    df[col] = df[col].map(yes_no_map)

# Doing the same thing for telephone sevices
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')
df['MultipleLines'] = df['MultipleLines'].map(yes_no_map)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,HasInternet
0,7590-VHVEG,Female,0,1,0,1,0,0,DSL,0,...,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,1
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,...,0,0,0,One year,0,Mailed check,56.95,1889.5,0,1
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,...,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,1
3,7795-CFOCW,Male,0,0,0,45,0,0,DSL,1,...,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,1
4,9237-HQITU,Female,0,0,0,2,1,0,Fiber optic,0,...,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,1,1,24,1,1,DSL,1,...,1,1,1,One year,1,Mailed check,84.80,1990.5,0,1
7039,2234-XADUH,Female,0,1,1,72,1,1,Fiber optic,0,...,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0,1
7040,4801-JZAZL,Female,0,1,1,11,0,0,DSL,1,...,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0,1
7041,8361-LTMKD,Male,1,1,0,4,1,1,Fiber optic,0,...,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1,1


#### Since customer id doesn't have any real use for churn prediction we can remove that column

In [140]:
df = df.drop('customerID', axis = 1)

In [141]:
df.to_csv('Data/Processed/ChurnModelling_imputed.csv', index=False)